# Ad Hoc в SkyTaxi

In [ ]:
# импортируем все необходимые библиотеки

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [ ]:
df        = pd.read_csv('C:/Users/computer/Documents/AD3.0/Финальные кейсы/Кейс 1. Исследование в такси/Data_Skytaxi.csv', sep=';')
df_city   = pd.read_csv('C:/Users/computer/Documents/AD3.0/Финальные кейсы/Кейс 1. Исследование в такси/city_dict.csv', sep=';')
df_tariff = pd.read_csv('C:/Users/computer/Documents/AD3.0/Финальные кейсы/Кейс 1. Исследование в такси/tariff_dict.csv', sep=';')

In [ ]:
# изучаем данные
df.head()

,order_id,id_tariff,id_city,order_time,assign_time,arrive_time,ride_finish_time
0,1146020,111,13,04.08.2022 7:12,NaN,NaN,NaN
1,1146340,114,11,02.08.2022 19:12,02.08.2022 19:19,02.08.2022 19:26,02.08.2022 19:50
2,1142747,112,13,01.08.2022 13:47,01.08.2022 13:53,01.08.2022 13:59,01.08.2022 14:16
3,1137227,111,17,03.08.2022 15:45,NaN,NaN,NaN
4,1137156,111,13,02.08.2022 15:00,02.08.2022 15:02,02.08.2022 15:11,02.08.2022 15:26


In [ ]:
#объединяем таблицы
df = df.merge(df_city, on = 'id_city')
df = df.merge(df_tariff, on = 'id_tariff')